In [ ]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)
import os 
path = '/content/drive/My Drive/project'
os.chdir(path)

import numpy as np 
from numpy.core.fromnumeric import shape
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device: ", device)

Device:  cuda


<h1> data stuff </h1>


In [ ]:
!python3 data.py

Intel MKL FATAL ERROR: Cannot load /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch_cpu.so.


In [ ]:
class data_load(Dataset):
    def __init__(self,train_seq,target_seq):
        self.train_seq = train_seq
        self.target_seq = target_seq
    def __len__(self):
        return len(self.train_seq)
    def __getitem__(self,idx):
        dat = self.train_seq[idx]
        gt = self.target_seq[idx]
        return dat,gt    


def get_features(features, gene):
    return (features + gene) % 3

def get_data(ref_file, data_file, Nref, Ntrain, Ntest, M, num_chunks):

    ref_panel = torch.load(ref_file)
    phase_data = torch.load(data_file)
    print(ref_panel.shape, phase_data.shape)
    my_phase_data = None
    for chunk in range(num_chunks):

        if my_phase_data is None:
            my_phase_data = phase_data[:, 0:500]
        else:
            my_phase_data = torch.cat((my_phase_data, phase_data[:, chunk * 500 : (chunk + 1)*500]))
    print(my_phase_data.shape,"my phase")
    target_seq = my_phase_data[:, 1:]
    my_phase_data = my_phase_data[:,:-1]
    print(phase_data.shape,"phase_data")
    phase_seq = torch.zeros((num_chunks * Ntrain, M - 1, Nref))

    for i in range(phase_seq.shape[0]):
        ind = i // Ntrain
        for j in range(M - 1):
            phase_seq[i, j, :] = get_features(ref_panel[:, M*ind + j], phase_data[i % Ntrain, M*ind + j])-1

    print(phase_seq.shape, target_seq.shape)

    return phase_seq, target_seq


In [ ]:
torch.manual_seed(0)
Nref = 200
Ntrain = 200
Ntest = 100
M = 500
F = 100
num_chunks = 1
# max_len = 100
ref_panel_file = "data/ref_panel_10000.t"
train_data_file = "data/train_data_10000.t"
test_data_file = "data/test_data_10000.t"


In [ ]:
train_seq, target_seq = get_data(ref_panel_file, train_data_file, Nref, Ntrain, Ntest, M, num_chunks)
#val_seq, target_seq_val = get_data(ref_panel_file, test_data_file, Nref, Ntrain, Ntest, M, num_chunks)
train_data = data_load(train_seq,target_seq)
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

torch.Size([200, 10000]) torch.Size([200, 10000])
torch.Size([200, 500]) my phase
torch.Size([200, 10000]) phase_data
torch.Size([200, 499, 200]) torch.Size([200, 499])


<h1> model stuff </h1>

In [ ]:
def accuracy(lst1, lst2):
    n = len(lst1)
    return np.sum(lst1 == lst2)/n

def accuracy_torch(lst1, lst2):
    n = len(lst1)
    return torch.sum(lst1 == lst2)/n

class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, embed_size=64):
        super(Model, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embed_layer = nn.Linear(input_size, embed_size)
        # self.rnn = nn.RNN(embed_size, hidden_dim, n_layers, batch_first=True)   
        self.rnn = nn.LSTM(embed_size, hidden_dim, n_layers, batch_first=True,bidirectional=False)
        self.fc = nn.Linear(1*hidden_dim, output_size)
        
        self.sigmoid = nn.LogSoftmax()
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # fvefrdsc
        
        batch_size = x.size(0)
        # Initializing hidden state for first input using method defined below
        # hidden = self.init_hidden(batch_size)

        x = self.embed_layer(x)
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x)
        # print(out.shape,"mmm")
        # out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, 1*self.hidden_dim)
        # print(out.shape,"dde")
        out = self.fc(out)
        out = self.relu(out)
        
        return out, hidden
        

In [ ]:
def sample_val(model, out_len, start, chunk):

    m = len(start)
    train_seq = torch.zeros((1, m, Nref))

    for i in range(train_seq.shape[0]):
        for j in range(m):
            train_seq[i, j, :] = get_features(ref_panel[:, M * chunk + j], start[j]) - 1

    model.eval()
    ind = len(start)
    dosages = []
    for _ in range(out_len - len(start)):
        train_seq = train_seq.to("cuda")
        out, hidden = model(train_seq)
        prob = nn.functional.softmax(out[-1], dim=0).data
        dosages.append(prob[0] * 0.0 + prob[1] * 1.0 + prob[2] * 2.0)
        # print(prob)
        gene = torch.max(prob, dim=0)[1].item()

        start.append(gene)
        feat = get_features(ref_panel[:, M * chunk + ind], gene)
        feat = feat.reshape(1, 1, len(feat)).to("cuda") - 1
        ind += 1
        train_seq = torch.cat((train_seq, feat), dim=1)
    
    dosages = [i.item() for i in dosages]
    print(dosages)
    return torch.tensor(start), dosages


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import r2_score

def validate(model,test_data, n_val, F, num_chunks=num_chunks):  
    total_acc = 0.0
    total_r2 = 0.0
    c = 0
    i = 0

    for data in test_data:
        i +=1
        print("dataa", data.shape)
        dat = data.tolist()
        local_acc = 0.0
        local_r2 = 0.0
        for chunk in (range(num_chunks)):
          inp_data = dat[M * chunk : M * chunk + F]
          generated, dosages = sample_val(model, M, inp_data, chunk)
          # print(generated)
          acc = accuracy(np.array(generated[F:]), np.array(dat[M * chunk + F: M * (chunk + 1)]))
          # r2 = r2_score(np.array(dat[M * chunk + F: M * (chunk + 1)]), np.array(dosages))
          r2 = r2_score(np.random.randint(0, 3, (M-F)), np.array(dosages))
          # print(generated, dat)
          local_acc += acc
          local_r2 += r2
        local_acc /= num_chunks
        local_r2 /= num_chunks
        print(c, "Accuracy: ", local_acc)
        print(c, "Avg R2:", local_r2)
        c += 1
        total_acc += local_acc
        total_r2 += local_r2
        if i == n_val or i == len(test_data):
          break
    print("Average Acc: ", total_acc/i)
    print("Average R2:", total_r2/i)
    return total_acc/i


In [ ]:
# from model import Model
torch.manual_seed(0)
model = Model(input_size=Nref, output_size=3, hidden_dim=32, n_layers=1, embed_size=32)
model = model.to(device)

n_epochs = 200
lr=1e-3

n_val = 1
test_data = torch.load("data/test_data_1kg.t")
ref_panel = torch.load(ref_panel_file)

criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr,betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

In [ ]:
print(device)
# optimizer.to(device)
# Training Run

val_acc = -1

for epoch in range(1, n_epochs + 1):
    model.train()
    running_loss, lo = 0.0, 0
    for mat,gt in tqdm(train_dataloader):
        optimizer.zero_grad()
        mat = mat.to(device)
        gt = gt.to(device)
        output, hidden = model(mat)
        loss = criterion(output, gt.contiguous().view(-1).long())
        prob = nn.functional.softmax(output, dim=0).data
        gene = torch.max(prob, dim=1)[1]
        acc = accuracy(np.array(gene.cpu().detach()), np.array(gt.contiguous().view(-1).long().cpu().detach()))
        # acc = accuracy_torch(gene, gt.contiguous().view(-1)).item()
        lo += acc

        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
        #scheduler.step()

    if epoch%10 == 1:
        cur_val_acc = validate(model,test_data,n_val)
        print("val acc: ", cur_val_acc)
        if cur_val_acc > val_acc:
            print('saving in epoch', epoch)
            val_acc = cur_val_acc
            print(f'new val acc: {val_acc}')
            torch.save(model, "model_best_lstm_art.pth")

    if epoch%1 == 0:
        print("train acc",lo/len(train_dataloader))
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("train Loss: {:.4f}".format(loss.item()))
        print("val Acc: ", val_acc)

In [ ]:
test_data = torch.load("data/test_data_1kg.t")
# model_best = torch.load("model_best_lstm_94_1kg.pth")
model_best = torch.load("model_62.pth")
for F in [100]:
  final_val_acc = validate(model_best,test_data,len(test_data), F=F, num_chunks=1)
  print(final_val_acc)

dataa torch.Size([10000])
[0.8504234552383423, 1.2164604663848877, 1.1229243278503418, 1.766209363937378, 0.8622874617576599, 0.874392032623291, 1.5882911682128906, 0.6414315104484558, 0.0830097571015358, 0.25102925300598145, 1.9052636623382568, 1.3642237186431885, 0.7462375164031982, 0.14893363416194916, 1.5016674995422363, 0.42242151498794556, 1.9465006589889526, 0.6122291684150696, 0.7077105045318604, 1.68416428565979, 0.12826655805110931, 1.8862810134887695, 0.5964319705963135, 1.3950039148330688, 1.982553482055664, 1.9250726699829102, 0.7456279397010803, 0.26129046082496643, 1.9481264352798462, 0.82900071144104, 1.6233117580413818, 1.060192346572876, 1.7314257621765137, 0.7958199977874756, 1.5867650508880615, 0.9363148212432861, 1.8453024625778198, 1.3563956022262573, 1.635040521621704, 0.3212730288505554, 0.3933440148830414, 0.9829307794570923, 1.8138542175292969, 1.8182430267333984, 0.21424923837184906, 1.9242627620697021, 0.9553394317626953, 1.7158279418945312, 0.65474772453308

KeyboardInterrupt: ignored

In [ ]:
#lstm _ 5  0.6187999999999999
0.621225, 0.6254333333333334, 0.6212000000000001, 0.6289000000000001 
0.9548500000000001, 0.9601999999999999, 0.9643999999999998, 0.9701999999999993

In [ ]:

def get_features(features, gene):
    return (features + gene) % 3

ref_panel = torch.load("data/ref_panel_10000.t")
Nref = 200
num_chunks = 5
M = 500

def sample(model, out_len, start, chunk):

    m = len(start)
    train_seq = torch.zeros((1, m, Nref))

    for i in range(train_seq.shape[0]):
        for j in range(m):
            train_seq[i, j, :] = get_features(ref_panel[:, M * chunk + j], start[j]) - 1

    model.eval()
    ind = len(start)

    for _ in range(out_len - len(start)):
        train_seq = train_seq.to("cuda")
        out, hidden = model(train_seq)
        prob = nn.functional.softmax(out[-1], dim=0).data
        gene = torch.max(prob, dim=0)[1].item()

        start.append(gene)
        feat = get_features(ref_panel[:, M * chunk + ind], gene)
        feat = feat.reshape(1, 1, len(feat)).to("cuda") - 1
        ind += 1
        train_seq = torch.cat((train_seq, feat), dim=1)

    return torch.tensor(start)

model = torch.load("model_best.pth")
test_data = torch.load("data/test_data_10000.t")

def accuracy(lst1, lst2):
    n = len(lst1)
    return np.sum(lst1 == lst2)/n


# for data in test_data:
#     # print(data.shape)
#     dat = data.tolist()
#     inp_data = dat[0:100]
#     generated = sample(model, 500, inp_data)
#     acc = accuracy(np.array(generated[100:]), np.array(dat[100:]))
#     # print(generated, dat)
#     print(c, "Accuracy: ", acc)
#     c += 1
#     total_acc += acc
#     # break
#     # exit(0)
# print("Average Acc: ", total_acc/len(test_data))

total_acc = 0
c = 0
for data in test_data:
    # print(data.shape)
    dat = data.tolist()
    local_acc = 0.0
    for chunk in tqdm(range(num_chunks)):
      inp_data = dat[M * chunk : M * chunk + 100]
      generated = sample(model, 500, inp_data, chunk)
      acc = accuracy(np.array(generated[100:]), np.array(dat[M * chunk + 100: M * (chunk + 1)]))
      # print(generated, dat)
      local_acc += acc
    local_acc /= num_chunks
    print(c, "Accuracy: ", local_acc)
    c += 1
    total_acc += local_acc

print("Average Acc: ", total_acc/len(test_data))

In [ ]:
!python3 test.py

In [ ]:
!python3 data.py